In [1]:
import tqdm
# ^^^ pyforest auto-imports - don't write above this line
import numpy as np
import pandas as pd

from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, normalized_mutual_info_score
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.cluster import SpectralClustering

from sklearn.decomposition import PCA, SparsePCA, KernelPCA
from sklearn.manifold import TSNE

from rpy2.robjects import r, pandas2ri
from rpy2.robjects.vectors import StrVector

pandas2ri.activate()

import magic
import scprep

%matplotlib inline

# from sklearnex import patch_sklearn
# patch_sklearn()

import warnings

from sklearn.cluster import KMeans
from tqdm import tqdm

from deepimpute.multinet import MultiNet

2023-09-03 17:26:53.724013: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
def get_cluster_metrics(pred, labels):
    ari_res = []
    ami_res = []
    nmi_res = []

#     try:
#         r.assign("data", pred.T)
#         seurat = r('''
#             countsData = data.frame(data)
#             pbmc <- CreateSeuratObject(counts = countsData, project = "thal_single_cell", min.cells = 1, min.features = 1)
#             pbmc <- FindVariableFeatures(pbmc, selection.method = "vst", verbose=FALSE)
#             all.genes <- rownames(pbmc)
#             pbmc <- ScaleData(pbmc, features = all.genes, verbose=FALSE)
#             pbmc <- RunPCA(pbmc, verbose=FALSE)
#             pbmc <- FindNeighbors(pbmc, verbose=FALSE)
#             pbmc <- FindClusters(pbmc, verbose=FALSE)
#             Idents(pbmc)
#         ''')
#         ari_res.append(adjusted_rand_score(labels, seurat))
#         ami_res.append(adjusted_mutual_info_score(labels, seurat))
#         nmi_res.append(normalized_mutual_info_score(labels, seurat))
#     except Exception as e:
#         pass

    pred_ = KMeans(n_clusters=len(np.unique(labels)), random_state=42).fit_predict(pred)

    ari_res.append(adjusted_rand_score(labels, pred_))
    ami_res.append(adjusted_mutual_info_score(labels, pred_))
    nmi_res.append(normalized_mutual_info_score(labels, pred_))

    warnings.filterwarnings("error")

    affinities = ['cosine', 'linear', 'poly']

    for i in affinities:
        try:
            pred_ = SpectralClustering(
                n_clusters=len(np.unique(labels)), 
                random_state=42, 
                affinity=i
            ).fit_predict(pred)
            ari_res.append(adjusted_rand_score(labels, pred_))
            ami_res.append(adjusted_mutual_info_score(labels, pred_))
            nmi_res.append(normalized_mutual_info_score(labels, pred_))
        except:
            ari_res.append(0)
            ami_res.append(0)
            nmi_res.append(0)

    warnings.resetwarnings()
    
    return max(ari_res), max(ami_res), max(nmi_res)

In [3]:
def get_data(i):
    df = pd.read_csv('../data/{}/data.csv.gz'.format(i), index_col=0)
    tmp = np.sign(df)
    cols = (np.sum(tmp) > int((df.shape[0])*0.05))
    rows = (np.sum(tmp, axis=1) > int((df.shape[1])*0.05))
    df = df.loc[rows, cols]
    X = df.reset_index(drop=True)
    labels = df.index
    return X, labels

In [4]:
dir_list = !ls ../data/ | grep -v zip
dir_list

['baron',
 'bmcite',
 'brosens',
 'carey',
 'cbmc',
 'chang',
 'Fujii',
 'hcabm40k',
 'hrvatin',
 'jakel',
 'jiang',
 'loureiro',
 'manno',
 'mingyao',
 'pbmc3k',
 'Selewa',
 'Xu']

In [5]:
len(dir_list)

17

In [6]:
res = []
for i in tqdm(dir_list):
    X, labels = get_data(i)
    X = X.reset_index(drop=True)
    try:
        multinet = MultiNet() 
        multinet.fit(X)
        pred = multinet.predict(X)

        res.append(get_cluster_metrics(pred, labels))
    except: 
        res.append([-100, -100, -100, ])

  0%|          | 0/17 [00:00<?, ?it/s]

Using all the cores (20)
Input dataset is 1791 cells (rows) and 8669 genes (columns)
First 3 rows and columns:
   A1CF  AAAS  AACS
0     4     0     0
1     0     0     2
2     0     0     0
3072 genes selected for imputation
Net 0: 658 predictors, 512 targets
Net 1: 618 predictors, 512 targets
Net 2: 653 predictors, 512 targets
Net 3: 666 predictors, 512 targets
Net 4: 653 predictors, 512 targets
Net 5: 633 predictors, 512 targets
Normalization
Building network
[{'type': 'dense', 'neurons': 256, 'activation': 'relu'}, {'type': 'dropout', 'rate': 0.2}]


2023-09-03 17:27:05.083881: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-09-03 17:27:05.084404: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-09-03 17:27:05.221879: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-03 17:27:05.221965: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3080 computeCapability: 8.6
coreClock: 1.71GHz coreCount: 68 deviceMemorySize: 9.77GiB deviceMemoryBandwidth: 707.88GiB/s
2023-09-03 17:27:05.221994: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-09-03 17:27:05.224241: I tensorflow/stream_executor/platform/def

Fitting with 1791 cells
Epoch 1/500


2023-09-03 17:27:05.442457: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-09-03 17:27:05.443179: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3600000000 Hz


...

64/64 [==============================] - 0s 6ms/step - loss: 0.6399 - dense_237_loss: 0.1059 - dense_238_loss: 0.0879 - dense_239_loss: 0.0822 - dense_240_loss: 0.0906 - dense_241_loss: 0.0975 - dense_242_loss: 0.0830 - dense_243_loss: 0.0927 - val_loss: 0.6228 - val_dense_237_loss: 0.0975 - val_dense_238_loss: 0.0879 - val_dense_239_loss: 0.0838 - val_dense_240_loss: 0.0862 - val_dense_241_loss: 0.0941 - val_dense_242_loss: 0.0816 - val_dense_243_loss: 0.0916
Epoch 35/500
64/64 [==============================] - 0s 7ms/step - loss: 0.6192 - dense_237_loss: 0.1027 - dense_238_loss: 0.0843 - dense_239_loss: 0.0808 - dense_240_loss: 0.0868 - dense_241_loss: 0.0941 - dense_242_loss: 0.0813 - dense_243_loss: 0.0893 - val_loss: 0.6182 - val_dense_237_loss: 0.0972 - val_dense_238_loss: 0.0878 - val_dense_239_loss: 0.0814 - val_dense_240_loss: 0.0858 - val_dense_241_loss: 0.0936 - val_dense_242_loss: 0.0818 - val_dense_243_loss: 0.0906
Epoch 36/500
64/64 [==============================] - 0s 

64/64 [==============================] - 0s 6ms/step - loss: 0.5603 - dense_237_loss: 0.0922 - dense_238_loss: 0.0772 - dense_239_loss: 0.0724 - dense_240_loss: 0.0794 - dense_241_loss: 0.0850 - dense_242_loss: 0.0739 - dense_243_loss: 0.0803 - val_loss: 0.5970 - val_dense_237_loss: 0.0943 - val_dense_238_loss: 0.0857 - val_dense_239_loss: 0.0779 - val_dense_240_loss: 0.0827 - val_dense_241_loss: 0.0897 - val_dense_242_loss: 0.0786 - val_dense_243_loss: 0.0881
Epoch 52/500
64/64 [==============================] - 0s 6ms/step - loss: 0.5588 - dense_237_loss: 0.0910 - dense_238_loss: 0.0770 - dense_239_loss: 0.0723 - dense_240_loss: 0.0791 - dense_241_loss: 0.0848 - dense_242_loss: 0.0730 - dense_243_loss: 0.0816 - val_loss: 0.5879 - val_dense_237_loss: 0.0930 - val_dense_238_loss: 0.0832 - val_dense_239_loss: 0.0775 - val_dense_240_loss: 0.0822 - val_dense_241_loss: 0.0887 - val_dense_242_loss: 0.0771 - val_dense_243_loss: 0.0863
Epoch 53/500
64/64 [==============================] - 0s 

64/64 [==============================] - 0s 6ms/step - loss: 0.5221 - dense_237_loss: 0.0850 - dense_238_loss: 0.0714 - dense_239_loss: 0.0678 - dense_240_loss: 0.0742 - dense_241_loss: 0.0791 - dense_242_loss: 0.0686 - dense_243_loss: 0.0760 - val_loss: 0.5782 - val_dense_237_loss: 0.0904 - val_dense_238_loss: 0.0812 - val_dense_239_loss: 0.0760 - val_dense_240_loss: 0.0800 - val_dense_241_loss: 0.0906 - val_dense_242_loss: 0.0756 - val_dense_243_loss: 0.0843
Epoch 69/500
64/64 [==============================] - 0s 6ms/step - loss: 0.5248 - dense_237_loss: 0.0852 - dense_238_loss: 0.0719 - dense_239_loss: 0.0676 - dense_240_loss: 0.0749 - dense_241_loss: 0.0805 - dense_242_loss: 0.0688 - dense_243_loss: 0.0759 - val_loss: 0.5717 - val_dense_237_loss: 0.0900 - val_dense_238_loss: 0.0807 - val_dense_239_loss: 0.0754 - val_dense_240_loss: 0.0797 - val_dense_241_loss: 0.0867 - val_dense_242_loss: 0.0749 - val_dense_243_loss: 0.0842
Epoch 70/500
64/64 [==============================] - 0s 

64/64 [==============================] - 0s 7ms/step - loss: 0.4882 - dense_237_loss: 0.0795 - dense_238_loss: 0.0666 - dense_239_loss: 0.0630 - dense_240_loss: 0.0701 - dense_241_loss: 0.0751 - dense_242_loss: 0.0636 - dense_243_loss: 0.0703 - val_loss: 0.5678 - val_dense_237_loss: 0.0897 - val_dense_238_loss: 0.0799 - val_dense_239_loss: 0.0744 - val_dense_240_loss: 0.0786 - val_dense_241_loss: 0.0865 - val_dense_242_loss: 0.0750 - val_dense_243_loss: 0.0837
Epoch 86/500
64/64 [==============================] - 0s 7ms/step - loss: 0.4928 - dense_237_loss: 0.0809 - dense_238_loss: 0.0676 - dense_239_loss: 0.0637 - dense_240_loss: 0.0701 - dense_241_loss: 0.0753 - dense_242_loss: 0.0644 - dense_243_loss: 0.0708 - val_loss: 0.5641 - val_dense_237_loss: 0.0894 - val_dense_238_loss: 0.0801 - val_dense_239_loss: 0.0743 - val_dense_240_loss: 0.0787 - val_dense_241_loss: 0.0851 - val_dense_242_loss: 0.0738 - val_dense_243_loss: 0.0827
Epoch 87/500
64/64 [==============================] - 0s 

100%|██████████| 17/17 [24:38<00:00, 86.99s/it]


In [7]:
pd.DataFrame(res)

,0,1,2
0,0.862336,0.769121,0.774502
1,0.243218,0.287293,0.288292
2,0.199072,0.355677,0.360277
3,0.374950,0.557663,0.559258
4,0.334800,0.524358,0.531968
5,0.121698,0.226555,0.251533
6,0.140330,0.228880,0.233954
7,0.024590,0.041959,0.044861
8,0.449229,0.688839,0.700287
9,0.358880,0.464266,0.466631
